In [1]:
import os
path = os.getcwd()

In [2]:
import time
import pandas as pd
from sklearn.cluster import KMeans

8-1 数据探索

In [3]:
datafile = path + '/data/data.xls'
processedfile = path + '/tmp/data_processed.xls'
data = pd.read_excel(datafile)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 10 columns):
肝气郁结证型系数     930 non-null float64
热毒蕴结证型系数     930 non-null float64
冲任失调证型系数     930 non-null float64
气血两虚证型系数     930 non-null float64
脾胃虚弱证型系数     930 non-null float64
肝肾阴虚证型系数     930 non-null float64
病程阶段         930 non-null object
TNM分期        930 non-null object
转移部位         930 non-null object
确诊后几年发现转移    930 non-null object
dtypes: float64(6), object(4)
memory usage: 72.7+ KB


In [4]:
data.head(10)

,肝气郁结证型系数,热毒蕴结证型系数,冲任失调证型系数,气血两虚证型系数,脾胃虚弱证型系数,肝肾阴虚证型系数,病程阶段,TNM分期,转移部位,确诊后几年发现转移
0,0.056,0.460,0.281,0.352,0.119,0.350,S4,H4,R1,J1
1,0.488,0.099,0.283,0.333,0.116,0.293,S4,H4,R1,J1
2,0.107,0.008,0.204,0.150,0.032,0.159,S4,H4,R2,J2
3,0.322,0.208,0.305,0.130,0.184,0.317,S4,H4,R2,J1
4,0.242,0.280,0.131,0.210,0.191,0.351,S4,H4,R2R5,J1
5,0.389,0.112,0.456,0.277,0.185,0.396,S4,H4,R3,J1
6,0.246,0.202,0.277,0.178,0.237,0.483,S4,H4,R1R3,J3
7,0.330,0.125,0.356,0.268,0.366,0.397,S4,H4,R1R2R3R5,J1
8,0.257,0.314,0.328,0.140,0.128,0.335,S4,H4,R2,J2
9,0.205,0.330,0.253,0.295,0.115,0.224,S4,H4,R2,J1


In [5]:
result = pd.DataFrame()

typelabel = {
    u'肝气郁结证型系数': 'A',
    u'热毒蕴结证型系数': 'B',
    u'冲任失调证型系数': 'C',
    u'气血两虚证型系数': 'D',
    u'脾胃虚弱证型系数': 'E',
    u'肝肾阴虚证型系数': 'F',
}
k = 4    # 聚类中心数

for key, item in typelabel.items():
    print(u"正在进行“%s”的聚类..." % key)
    # 进行聚类离散化
    kmodel = KMeans(n_clusters = k, n_jobs = 4)
    kmodel.fit(data[[key]].as_matrix())

    # 聚类中心
    r1 = pd.DataFrame(kmodel.cluster_centers_, columns = [item])
    # 分类统计
    r2 = pd.Series(kmodel.labels_).value_counts()
    r2 = pd.DataFrame(r2, columns = [item + 'n'])
    # 匹配聚类中心与类别
    r = pd.concat([r1, r2], axis = 1).sort_values(item)
    r.index = list(range(1, 5))
    # 用来计算相邻两列的均值，以此作为边界点
    r[item] = pd.Series.rolling(r[item], 2).mean()
    # 将NaN值转为0.0，不用fillna的原因是数值类型是float64
    r.loc[1, item] = 0.0
    result = result.append(r.T)
    
# 以ABCDEF排序
result = result.sort_index()
result.to_excel(processedfile)

正在进行“肝气郁结证型系数”的聚类...


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


正在进行“热毒蕴结证型系数”的聚类...
正在进行“冲任失调证型系数”的聚类...
正在进行“气血两虚证型系数”的聚类...
正在进行“脾胃虚弱证型系数”的聚类...
正在进行“肝肾阴虚证型系数”的聚类...


8-2 Apriori关联规则算法

In [6]:
# 自定义连接函数
def connect_string(x, ms):
    x = list(map(lambda i: sorted(i.split(ms)), x))
    r = []
    for i in range(len(x)):
        for j in range(i + 1, len(x)):
            if x[i][:-1] == x[j][:-1] and x[i][-1] != x[j][-1]:
                r.append(x[i][:-1] + sorted([x[j][-1], x[i][-1]]))
    return r

# 寻找关联规则函数
def find_rule(d, support, confidence, ms = u'---'):
    result = pd.DataFrame(index = ['support', 'confidence'])

    # 第一批支持度筛选
    support_series = 1.0 * d.sum() / len(d)

    column = list(support_series[support_series > support].index)
    k = 0

    while len(column) > 1:
        k = k + 1
        print(u"\n正在进行第%s次搜索..." % k)

        column = connect_string(column, ms)
        print(u"数目%s..." % len(column))
        index_lst = [ms.join(i) for i in column]

        # 新的支持度函数
        sf = lambda i: d[i].prod(axis = 1, numeric_only = True)
        # 计算连接后的支持度，开始筛选
        d_2 = pd.DataFrame(list(map(sf, column)), index = index_lst).T
        support_series_2 = 1.0 * d_2[index_lst].sum() / len(d)
        column = list(support_series_2[support_series_2 > support].index)

        support_series = support_series.append(support_series_2)
        column2 = []
        # 遍历所有可能的情况
        for i in column:
            i = i.split(ms)
            for j in range(len(i)):
                column2.append(i[:j] + i[j + 1:] + i[j:j + 1])

        # 置信度序列
        cofidence_series = pd.Series(index = [ms.join(i) for i in column2])

        for i in column2:
            cofidence_series[ms.join(i)] = support_series[ms.join(
                sorted(i))] / support_series[ms.join(i[:-1])]
        # 置信度筛选
        for i in cofidence_series[cofidence_series > confidence].index:
            result[i] = 0.0
            result[i]['confidence'] = cofidence_series[i]
            result[i]['support'] = support_series[ms.join(sorted(i.split(ms)))]

    result = result.T.sort_values(['confidence', 'support'], ascending = False)
    print(u"\nresult:")
    print(result)

    return result

In [11]:
inputfile = path + '/data/apriori.txt'
data_apriori = pd.read_csv(inputfile, header = None, dtype = object)
data_apriori.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 930 entries, 0 to 929
Data columns (total 7 columns):
0    930 non-null object
1    930 non-null object
2    930 non-null object
3    930 non-null object
4    930 non-null object
5    930 non-null object
6    930 non-null object
dtypes: object(7)
memory usage: 50.9+ KB


In [12]:
start = time.clock()
print(u"\n转换原始数据至0-1矩阵...")

# 0-1矩阵的转换
ct = lambda x: pd.Series(1, index = x)
b = list(map(ct, data_apriori.as_matrix()))
data_apriori = pd.DataFrame(b).fillna(0)

end = time.clock()
print(u"\n转换完毕，用时：%0.2f s" % (end - start))

# 删除中间变量b，节省内存
del b

# 定义支持度，置信度，连接符号
support = 0.06
confidence = 0.75
ms = '----'

start = time.clock()
print(u'\n开始搜索关联规则...')
find_rule(data_apriori, support, confidence, ms)
end = time.clock()
print(u'\n搜索完成，用时%0.2f s' % (end - start))


转换原始数据至0-1矩阵...


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  



转换完毕，用时：1.29 s

开始搜索关联规则...

正在进行第1次搜索...
数目276...

正在进行第2次搜索...
数目947...

正在进行第3次搜索...
数目41...

result:
                       support  confidence
A3----F4----H4        0.078495    0.879518
C3----F4----H4        0.075269    0.875000
B2----F4----H4        0.062366    0.794521
C2----E3----D2        0.092473    0.754386
D2----F3----H4----A2  0.062366    0.753247

搜索完成，用时4.92 s
